<a href="https://colab.research.google.com/github/kbee181756/DS-Unit-1-Sprint-2-Statistics/blob/master/Copy_of_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-30 02:55:04--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2020-01-30 02:55:04 (597 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [0]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

df = pd.read_csv('house-votes-84.data', 
                 header=None, 
                 names=column_headers,
                 na_values="?")

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
df = df.replace({'y':1, 'n': 0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [0]:
rep = df[df["party"] == 'republican']
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
dem = df[df["party"] == 'democrat']
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
rep_edu = rep['education']
rep_edu = rep[~np.isnan(rep_edu)]
rep_edu.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
14,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,NaN


In [0]:
rep_edu["education"].mean()

0.8709677419354839

In [0]:
dem_edu = dem['education']
dem_edu = dem[~np.isnan(dem_edu)]
dem_edu.head()

dem_edu['education'].mean()

0.14457831325301204

In [0]:
dem_edu_mean = dem_edu['education'].mean()
rep_edu_mean = rep_edu['education'].mean()

In [0]:
print(dem_edu_mean)
print(rep_edu_mean)

0.14457831325301204
0.8709677419354839


In [0]:
ttest_ind(rep['education'], dem['education'], nan_policy='omit')

Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)

In [0]:
immigration_results = ttest_ind(rep['immigration'], dem['immigration'], nan_policy='omit')
immigration_results
print('{:f}'.format(immigration_results[1]))

0.083302


In [0]:
right_to_sue_results = ttest_ind(rep['right-to-sue'], dem['right-to-sue'], nan_policy='omit')
right_to_sue_results
print(right_to_sue_results[1])
print('{:f}'.format(right_to_sue_results[1]))
print('{:.15f}'.format(right_to_sue_results[1]))


1.2278581709672758e-34
0.000000
0.000000000000000


In [0]:
aid_to_contras_results = ttest_ind(rep['aid-to-contras'], dem['aid-to-contras'], nan_policy='omit')
aid_to_contras_results
print(aid_to_contras_results[1])
print('{:f}'.format(aid_to_contras_results[1]))
print('{:.15f}'.format(aid_to_contras_results[1]))

2.82471841372357e-54
0.000000
0.000000000000000


In [0]:
mx_missile_results = ttest_ind(rep['mx-missile'], dem['mx-missile'], nan_policy='omit')
mx_missile_results
print(mx_missile_results[1])
print('{:f}'.format(mx_missile_results[1]))
print('{:.15f}'.format(mx_missile_results[1]))

5.03079265310811e-47
0.000000
0.000000000000000


In [0]:
duty_free_results = ttest_ind(rep['duty-free'], dem['duty-free'], nan_policy='omit')
duty_free_results
print(duty_free_results[1])
print('{:f}'.format(duty_free_results[1]))
print('{:.15f}'.format(duty_free_results[1]))

5.997697174347365e-32
0.000000
0.000000000000000


In [0]:
synfuels_results = ttest_ind(rep['synfuels'], dem['synfuels'], nan_policy='omit')
synfuels_results
print(synfuels_results[1])
print('{:f}'.format(synfuels_results[1]))
print('{:.15f}'.format(synfuels_results[1]))

1.5759322301054064e-15
0.000000
0.000000000000002


In [0]:
anti_statellite_ban_results = ttest_ind(rep['anti-satellite-ban'], dem['anti-satellite-ban'], nan_policy='omit')
anti_statellite_ban_results
print(anti_statellite_ban_results[1])
print('{:f}'.format(anti_statellite_ban_results[1]))
print('{:.15f}'.format(anti_statellite_ban_results[1]))

8.521033017443867e-31
0.000000
0.000000000000000


In [0]:
religious_groups_results = ttest_ind(rep['religious-groups'], dem['religious-groups'], nan_policy='omit')
religious_groups_results
print(religious_groups_results[1])
print('{:f}'.format(religious_groups_results[1]))
print('{:.15f}'.format(religious_groups_results[1]))

2.3936722520597287e-20
0.000000
0.000000000000000


In [0]:
crime_results = ttest_ind(rep['crime'], dem['crime'], nan_policy='omit')
crime_results
print(crime_results[1])
print('{:f}'.format(crime_results[1]))
print('{:.15f}'.format(crime_results[1]))

9.952342705606092e-47
0.000000
0.000000000000000
